In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim 
import torch.nn.functional as F
import nltk 
import random 
import numpy as np
from collections import Counter
flatten = lambda l: [item for sublist in l for item in sublist]
random.seed(1024)

In [3]:
print(torch.__version__)
print(nltk.__version__)

0.4.1
3.2.3


In [25]:
USE_CUDA = torch.cuda.is_available()
gpus = [0]
#torch.cuda.set_device(gpus[0])

FloatTensor = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if USE_CUDA else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if USE_CUDA else torch.ByteTensor

In [4]:
def getBatch(batch_size, train_data):
    random.shuffle(train_data)
    sindex = 0           # start index
    eindex = batch_size  # end index
    while eindex < len(train_data):
        batch = train_data[sindex: eindex]
        temp = eindex
        eindex = eindex + batch_size
        sindex = temp
        yield batch
    
    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch
    

In [5]:
# 
def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w: word2index[w] 
                    if word2index.get(w) is not None 
                    else word2index["<UNK>"], seq))
    return Variable(LongTensor(idxs))

def prepare_word(word, word2index):
    return Variable(LongTensor([word2index[word]]) 
                    if word2index.get(word) is not None 
                    else LongTensor([word2index["<UNK>"]]))

## Data load and Preprocessing

### Load corpus: Gutenberg corpus

In [6]:
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\yu\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [7]:
nltk.corpus.gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [8]:
# Corpus load 
corpus = list(nltk.corpus.gutenberg.sents('melville-moby_dick.txt'))[:100] # sampling sentences for test
# 대문자를 소문자로 
corpus = [[word.lower() for word in sent] for sent in corpus]

### Extract Stopwords from unigram distribution's tails

In [9]:
#상위, 하위 빈도 단어 1% 제거
word_count = Counter(flatten(corpus))
border = int(len(word_count)*0.01)   

In [10]:
stopwords = word_count.most_common()[:border] + list(reversed(word_count.most_common()))[:border]

In [11]:
stopwords = [s[0] for s in stopwords]

In [12]:
stopwords

[',', '.', 'the', 'of', 'and', 'man', 'artificial', 'civitas', '--(', 'state']

### Build vocab

In [14]:
vocab = list(set(flatten(corpus))- set(stopwords))
vocab.append('<UNK>')

In [15]:
print(len(set(flatten(corpus))), len(vocab))

592 583


In [16]:
word2index = {'<UNK>': 0}

# word2index - word : index 형태 
for vo in vocab:
    if word2index.get(vo) is None:
        word2index[vo] = len(word2index)
        
# index2word - index : word 형태로 변경 
index2word = {v:k for k, v in word2index.items()}

### Prepare train data

window data example

![](https://nbviewer.jupyter.org/github/DSKSD/DeepNLP-models-Pytorch/blob/master/images/01.skipgram-prepare-data.png)

borrowed image from http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/

In [18]:
WINDOW_SIZE = 3
windows = flatten([list(nltk.ngrams(['<DUMMY>'] * WINDOW_SIZE + c + ['<DUMMY>'] * WINDOW_SIZE, 
                                    WINDOW_SIZE * 2 + 1)) for c in corpus])

In [19]:
windows[0]

('<DUMMY>', '<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by')

In [20]:
train_data = []

for window in windows:
    for i in range(WINDOW_SIZE *2 +1):
        if i == WINDOW_SIZE or window[i] == '<DUMMY>':
            continue
        train_data.append((window[WINDOW_SIZE], window[i]))

print(train_data[:WINDOW_SIZE * 2])

[('[', 'moby'), ('[', 'dick'), ('[', 'by'), ('moby', '['), ('moby', 'dick'), ('moby', 'by')]


In [21]:
X_p = []
y_p = []

In [22]:
train_data[0]

('[', 'moby')

In [26]:
for tr in train_data:
    X_p.append(prepare_word(tr[0], word2index).view(1, -1))
    y_p.append(prepare_word(tr[1], word2index).view(1, -1))

In [27]:
train_data = list(zip(X_p, y_p))

In [28]:
len(train_data)

7606

### Modeling

![](https://nbviewer.jupyter.org/github/DSKSD/DeepNLP-models-Pytorch/blob/master/images/01.skipgram-objective.png)

borrowed image from http://web.stanford.edu/class/cs224n/lectures/cs224n-2017-lecture2.pdf

In [29]:
class Skipgram(nn.Module):
    
    def __init__(self, vocab_size, projection_dim):
        super(Skipgram,self).__init__()
        self.embedding_v = nn.Embedding(vocab_size, projection_dim)
        self.embedding_u = nn.Embedding(vocab_size, projection_dim)

        # weight initialization
        self.embedding_v.weight.data.uniform_(-1, 1) # init
        self.embedding_u.weight.data.uniform_(0, 0) # init
        #self.out = nn.Linear(projection_dim,vocab_size)
        
    def forward(self, center_words, target_words, outer_words):
        center_embeds = self.embedding_v(center_words) # B x 1 x D
        target_embeds = self.embedding_u(target_words) # B x 1 x D
        outer_embeds = self.embedding_u(outer_words) # B x V x D
        
        scores = target_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2) # Bx1xD * BxDx1 => Bx1
        norm_scores = outer_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2) # BxVxD * BxDx1 => BxV
        
        # log-softmax
        nll = -torch.mean(torch.log(torch.exp(scores)/torch.sum(torch.exp(norm_scores), 1).unsqueeze(1))) 
        
        return nll # negative log likelihood
    
    def prediction(self, inputs):
        embeds = self.embedding_v(inputs)
        
        return embeds

### Train

In [30]:
EMBEDDING_SIZE = 30
BATCH_SIZE = 256
EPOCH =100

In [31]:
losses = []
model = Skipgram(len(word2index), EMBEDDING_SIZE)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [36]:
for epoch in range(EPOCH):
    for i, batch in enumerate(getBatch(BATCH_SIZE, train_data)):
        inputs, targets = zip(*batch)
        
        inputs = torch.cat(inputs)
        targets = torch.cat(targets)
        vocabs = prepare_sequence(list(vocab), word2index).expand(inputs.size(0), len(vocab))  # B x V
        
        loss = model(inputs, targets, vocabs)
        
        # Back propagation
        model.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.data[0])
       
        
    if epoch % 10 ==0:
        print("Epoch : %d, mean_loss : %.02f" % (epoch,np.mean(losses)))
        losses = []

C:\Users\yu\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  from ipykernel import kernelapp as app


Epoch : 0, mean_loss : 6.15
Epoch : 10, mean_loss : 4.36
Epoch : 20, mean_loss : 3.48
Epoch : 30, mean_loss : 3.31
Epoch : 40, mean_loss : 3.26
Epoch : 50, mean_loss : 3.24
Epoch : 60, mean_loss : 3.22
Epoch : 70, mean_loss : 3.21
Epoch : 80, mean_loss : 3.21
Epoch : 90, mean_loss : 3.20


### Test

In [40]:
def word_similarity(target, vocab):
    target_V = model.prediction(prepare_word(target, word2index))
    similarities = []
    
    for i in range(len(vocab)):
        if vocab[i] == target:
            continue
            
        vector = model.prediction(prepare_word(list(vocab)[i], word2index))
        cosine_sim = F.cosine_similarity(target_V, vector).data.tolist()[0]
        similarities.append([vocab[i], cosine_sim])
    # sort by similarity
    return sorted(similarities, key=lambda x: x[1], reverse=True)[:10]

In [41]:
test = random.choice(list(vocab))
test

'ceti'

In [42]:
word_similarity(test, vocab)

[['english', 0.71509850025177],
 ['however', 0.713376522064209],
 ['statements', 0.6277019381523132],
 ['sperma', 0.621660053730011],
 ['icelandic', 0.6078787446022034],
 ['authentic', 0.5876720547676086],
 ['fat', 0.5666032433509827],
 ['890', 0.5662860870361328],
 ['piggledy', 0.5464634299278259],
 ['cetus', 0.5460925102233887]]